In [1]:
import tensorflow as tf
import tf_retinanet
import os

In [2]:
dataset = tf_retinanet.data.read(os.path.join('data', 'dataset.record-?????-of-00010'))

In [3]:
for image, bboxes, labels in dataset.take(1):
    print(image)

tf.Tensor(
[[[[   9.828194     8.9881935    6.087194 ]
   [  10.845081    10.005081     7.104081 ]
   [  11.344469    10.504469     7.603469 ]
   ...
   [ 127.061      118.221      105.32     ]
   [ 127.061      118.221      105.32     ]
   [ 127.061      118.221      105.32     ]]

  [[  11.6135      10.7735       7.8725   ]
   [  11.061       10.221        7.32     ]
   [   9.736406     8.896406     5.995406 ]
   ...
   [ 128.6135     119.7735     106.8725   ]
   [ 128.6135     119.7735     106.8725   ]
   [ 128.6135     119.7735     106.8725   ]]

  [[  10.4735       9.6335       6.7325   ]
   [  10.245594     9.405594     6.504594 ]
   [   9.061        8.221        5.32     ]
   ...
   [ 129.6485     120.8085     107.9075   ]
   [ 129.6485     120.8085     107.9075   ]
   [ 129.6485     120.8085     107.9075   ]]

  ...

  [[  27.609625    43.683094    42.051125 ]
   [  31.109781    46.531876    43.569187 ]
   [  44.52147     57.896782    53.645782 ]
   ...
   [ -95.0515    -102.89

In [4]:
image.shape

TensorShape([1, 200, 295, 3])

In [5]:
bboxes.shape

TensorShape([1, 11412, 5])

In [6]:
import tensorflow as tf
from tf_retinanet.builders import application_builder
from tf_retinanet.builders import loss_builder

In [7]:
model = application_builder.build('mobilenet_v1', num_classes=1, num_anchors=9)

In [8]:
model.compile(optiomizer=tf.keras.optimizers.Adam(lr=1e-5),
             loss=[loss_builder.build_smooth_l1_loss(), loss_builder.build_focal_loss()])

In [9]:
predictions = model(image)

In [10]:
smooth = loss_builder.build_smooth_l1_loss()
focal = loss_builder.build_focal_loss()

In [11]:
bboxes.shape, predictions[0].shape

(TensorShape([1, 11412, 5]), TensorShape([1, 10719, 4]))

In [12]:
smooth(bboxes, predictions[0])

<tf.Tensor: id=5379, shape=(), dtype=float32, numpy=2.876294>

In [13]:
focal(labels, predictions[1])

<tf.Tensor: id=5444, shape=(), dtype=float32, numpy=12.528548>

In [14]:
import tensorflow as tf

In [15]:
@tf.function
def train(image, bboxes, classes):
    with tf.GradientTape() as tape:
        pred_bboxes, pred_classes = model(image)
    

In [16]:
model.fit(image, [bboxes, labels])

Train on 1 samples
1/1 [==============================] - 12s 12s/sample - loss: 15.4412 - localization_loss: 2.8761 - classification_loss: 12.5651


In [20]:
dataset = dataset.shuffle(100)

In [21]:
def input_fn():
    for image, bboxes, classes in dataset:
        yield (image, [bboxes, classes])

In [22]:
gen = iter(input_fn())
model.fit_generator(input_fn(), steps_per_epoch=100)

100/100 [==============================] - 37s 369ms/step - loss: 5.7856 - localization_loss: 2.4307 - classification_loss: 3.3549


In [24]:
from time import time

In [30]:
st = time()
box, classmodel(image)
print(time() - st)

0.0638282299041748
